In [1]:
import configparser
import pandas as pd
from utils.geocode import address_to_coords
from utils.filtering_for_addr import AddressFilter
from utils.apis import get_hospitals_by_condition, get_real_time_bed_info
from utils.direction import calculate_travel_time_and_sort

def main(address, conditions_korean):
    """
    병원 데이터 처리 메인 함수
    :param address: 사용자 입력 주소 (도로명 주소 형식)
    :param conditions: 병원 필터링 조건 목록 (예: ["MKioskTy8", "MKioskTy10"])
    """
    # 설정 파일 로드
    config = configparser.ConfigParser()
    config.read('C:/Users/user/Desktop/24-2/졸업프로젝트/project_ai/keys.config')

    # AddressFilter 인스턴스 생성
    address_filter = AddressFilter('C:/Users/user/Desktop/24-2/졸업프로젝트/project_ai/keys.config')

    # 1. 사용자 주소 -> 좌표 변환
    api_key = config['API_KEYS']['kakao_api_key']
    user_coords = address_to_coords(address, api_key)

    if "error" in user_coords:
        print(f"오류: {user_coords['error']}")
        return

    user_lat = user_coords["lat"]
    user_lon = user_coords["lon"]
    print(f"사용자 좌표: 위도 {user_lat}, 경도 {user_lon}")

    # 2. 병원 조건 설정
    stage1, stage2 = address.split()[:2]  # Stage1 = 시도, Stage2 = 시군구

    # 3. 병원 조건에 맞는 hpid 수집
    condition_mapping = {
        "조산산모": "MKioskTy8",
        "정신질환자": "MKioskTy9",
        "신생아": "MKioskTy10",
        "중증화상": "MKioskTy11"
    }
    conditions = [condition_mapping[cond] for cond in conditions_korean if cond in condition_mapping]
    if not conditions:
        print("유효한 조건이 없습니다. 조건명을 확인하세요.")
        return None
    
    hpid_list = get_hospitals_by_condition(stage1, stage2, conditions)
    if not hpid_list:
        print("조건에 맞는 병원이 없습니다.")
        return

    print(f"필터링된 병원 목록: {hpid_list}")

    # 4. 실시간 병상 정보 조회
    real_time_data = get_real_time_bed_info(stage1, stage2, hpid_list)
    if not real_time_data:
        print("실시간 병상 정보가 없습니다.")
        return

    # 5. 병상 정보 DataFrame 생성
    df = pd.DataFrame(real_time_data)

    # 6. enriched_df 생성 및 저장
    enriched_df = address_filter.enrich_filtered_df(df)

    # 7. 소요 시간 계산 및 정렬
    enriched_df = calculate_travel_time_and_sort(enriched_df, user_lat, user_lon)

    # 추가된 칼럼 확인
    if "distance_km" in enriched_df.columns and "travel_time_h" in enriched_df.columns:
        print("거리와 소요 시간이 추가된 데이터:")
        print(enriched_df[["distance_km", "travel_time_h", "travel_time_m", "travel_time_s"]].head())

    # 8. 결과 저장
    enriched_df.to_csv("enriched_filtered_df.csv", index=False, encoding="utf-8-sig")
    print("결과가 'enriched_filtered_df.csv' 파일로 저장되었습니다.")
    
    return enriched_df


In [2]:
if __name__ == "__main__":
    #조산산모(MKioskTy8), 정신질환자(MKioskTy9), 신생아(MKioskTy10), 중증화상(MKioskTy11)
    enriched_df = main(address="서울특별시 강남구 삼성로 212", 
                       conditions_korean= ["정신질환자"] )
    if enriched_df is not None:
        print(enriched_df.head(5))

사용자 좌표: 위도 37.4996157705058, 경도 127.065915414443
생성된 Redis 키: hospitals:서울특별시:강남구:MKioskTy9
저장된 hpid Redis 캐시 데이터: None
Redis 캐시 없음, API 호출 진행
필터링된 병원 목록: ['A1100015', 'A1100010']
생성된 Redis 키: real_time_bed_info:A1100015
저장된 응급실 Redis 캐시 데이터: None
Redis 캐시 없음, A1100015 데이터 API 호출 진행
생성된 Redis 키: real_time_bed_info:A1100010
저장된 응급실 Redis 캐시 데이터: None
Redis 캐시 없음, A1100010 데이터 API 호출 진행
외상센터 hpid 수집 완료: ['A1500002', 'A1600002', 'A1700004', 'A2100002', 'A2100040', 'A2200001', 'A2300001', 'A2400002', 'A2500001', 'A1100008']
거리와 소요 시간이 추가된 데이터:
   distance_km  travel_time_h  travel_time_m  travel_time_s
0        2.291              0             12             12
1        3.899              0             15             52
결과가 'enriched_filtered_df.csv' 파일로 저장되었습니다.
             dutyName                           dutyAddr            wgs84Lat  \
0  연세대학교의과대학 강남세브란스병원  서울특별시 강남구 언주로 211, 강남세브란스병원 (도곡동)  37.492806984645476   
1              삼성서울병원      서울특별시 강남구 일원로 81 (일원동, 삼성의료원)   37.48851613

In [3]:
enriched_df.columns

Index(['dutyName', 'dutyAddr', 'wgs84Lat', 'wgs84Lon', 'dutyTel3', 'hpid',
       'hv10', 'hv11', 'hv28', 'hv29', 'hv30', 'hv34', 'hv41', 'hv42', 'hv5',
       'hv6', 'hv7', 'hvamyn', 'hvangioayn', 'hvcrrtayn', 'hvctayn', 'hvec',
       'hvecmoayn', 'hvgc', 'hvhypoayn', 'hvicc', 'hvidate', 'hvincuayn',
       'hvmriayn', 'hvncc', 'hvoc', 'hvoxyayn', 'hvs01', 'hvs02', 'hvs03',
       'hvs04', 'hvs08', 'hvs12', 'hvs15', 'hvs17', 'hvs22', 'hvs25', 'hvs26',
       'hvs27', 'hvs28', 'hvs29', 'hvs30', 'hvs31', 'hvs32', 'hvs33', 'hvs34',
       'hvs35', 'hvs38', 'hvventiayn', 'hvventisoayn', 'phpid', 'rnum', 'hv2',
       'hv3', 'hv32', 'hv35', 'hv40', 'hvccc', 'hvs06', 'hvs07', 'hvs09',
       'hvs16', 'hvs18', 'hvs24', 'travelTime', 'distance_km', 'congestion',
       'travel_time_h', 'travel_time_m', 'travel_time_s'],
      dtype='object')

In [4]:
enriched_df[['dutyName','dutyAddr','dutyTel3', "distance_km",
             "travel_time_h", "travel_time_m", "travel_time_s", 'congestion','hvamyn']]

,dutyName,dutyAddr,dutyTel3,distance_km,travel_time_h,travel_time_m,travel_time_s,congestion,hvamyn
0,연세대학교의과대학 강남세브란스병원,"서울특별시 강남구 언주로 211, 강남세브란스병원 (도곡동)",02-2019-3333,2.291,0,12,12,서행,Y
1,삼성서울병원,"서울특별시 강남구 일원로 81 (일원동, 삼성의료원)",02-3410-2060,3.899,0,15,52,서행,Y
